In [1]:
#import snappy_utils
import os, sys
import json
sys.path.append("../")
import settings
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date, datetime
import zipfile
import time

### Create global variables

In [2]:
footprint = settings.footprint
sentinel_api_user = settings.sentinel_api_user
sentinel_api_key = settings.sentinel_api_key
data_path = settings.data_path

### Sentinelsat API wrapper class

In [4]:
class Sentinelsat_products:
    def __init__(self, date_start, date_finish, footprint=footprint, platformname="Sentinel-2"):
        self.date_start = date_start
        self.date_finish = date_finish
        self.platform = platformname
        self.api = SentinelAPI(sentinel_api_user, sentinel_api_key, 'https://scihub.copernicus.eu/dhus')
        #self.api.logger.setLevel(logging.DEBUG)
        self.wkt_footprint = footprint
        self.products = self.query_products(self.date_start, self.date_finish)
        self.downloaded_prods, self.retrieval_scheduled, self.failed_prods = {}, {}, {}

    def query_products(self, date_start, date_finish, platformname="Sentinel-2"):
        # search by polygon, time, and Hub query keywords
        products = self.api.query(self.wkt_footprint, area_relation='Contains',
                            date = (self.date_start, self.date_finish),
                            platformname = platformname)
        return products
    
    def filter_products(self, instrument, level, p_type, timeliness):
        removed_products = []
        for product_key in self.products:
            odata = self.api.get_product_odata(product_key, full=True)
            #print(odata)
            if self.platform == "Sentinel-3":
                product_instrument = odata["Instrument"]
                product_level = odata["Product level"]
                product_type = odata["Product type"]
                mission_type = odata["Mission type"]
                product_timeliness = odata["Timeliness Category"]
                #filter only from Level 1 OLCI instrument with NTC full resolution
                conditions = (
                (product_instrument == instrument) and (p_type in product_type) 
                and product_timeliness == timeliness and product_level == level
                            )
            if self.platform == "Sentinel-2":
                product_instrument = odata["Instrument"]
                product_level = odata["Processing level"]            
                conditions = (product_instrument == instrument) and (product_level == level)
            #keep list of pids of products that meet downloading conditions
            if conditions:
                #print(instrument, product_level, product_type)
                pass                
            else:
                removed_products.append(product_key)
        #remove products that didn't meet the download conditions
        for key in removed_products:
            del self.products[key]

    def download_products(self, delete_zip=True, lta_request_delay=600):
        total_products = len(self.products.keys())
        current_product = 1
        for key in self.products:
            print("-------------------------")
            print(current_product, "/", total_products)
            current_product += 1
            product_odata = self.api.get_product_odata(key, full=True)
            file_name = self.products[key]["filename"]
            file_date = self.products[key]["summary"][:16].split("Date: ")[1]
            print(file_date)
            download_path = os.path.join(data_path, file_date)
            if not os.path.exists(download_path):
                os.makedirs(download_path)
            # if it was downloaded before or is offline it won't download
            if file_name in os.listdir(download_path):
                print("Data from", file_date, "already downloaded")
                self.downloaded_prods[key] = self.products[key]
                continue
            if self.api.is_online(key) == False:
                print("Data from", file_date, "offline")
                if key in list(self.retrieval_scheduled.keys()):
                    print("LTA retrieval already triggered")
                    continue
                try:
                    lta_status_response = self.api._trigger_offline_retrieval(product_odata["url"])
                except Exception as e:
                    print("RAISED EXCEPTION:", str(e))
                    print("///////////////////////////")
                    self.failed_prods[key] = self.products[key]
                    continue
                if lta_status_response == 202:
                    print("Accepted for retrieval")
                    self.retrieval_scheduled[key] = self.products[key]
                    continue
                elif lta_status_response == 403:
                    while(lta_status_response == 403):
                        print("Requests exceed user quota. Retrying in %s seconds" % (lta_request_delay))
                        time.sleep(lta_request_delay)
                        lta_status_response = self.api._trigger_offline_retrieval(product_odata["url"])
                    if lta_status_response == 202:
                        print("Accepted for retrieval")
                        self.retrieval_scheduled[key] = self.products[key]
                continue
            try:
                download_info = self.api.download(key, directory_path=download_path)
                zip_path = download_info["path"]
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
                self.downloaded_prods[key] = self.products[key]
            except Exception as e:
                print("RAISED EXCEPTION:", str(e))
                print("/////////////")
                self.failed_prods[key] = self.products[key]
                continue
            if delete_zip:
                os.remove(zip_path)

### Generic data download code

In [ ]:
date1 = date(2015, 11, 1)
date2 = date(2016, 5, 31)

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-2A", p_type=None, timeliness=None)
datos.download_products()

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos.download_products()

### Download 2015-2016 data

In [ ]:
print("Starting datetime:", str(datetime.now()))

date1 = date(2015, 11, 1)
date2 = date(2016, 5, 31)

datos__2016_2015 = Sentinelsat_products(date1, date2, footprint=footprint)
datos__2016_2015.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos__2016_2015.download_products()
print("Finish datetime: ", str(datetime.now()))

### Download 2017-2018 data

In [ ]:
print("Starting datetime:", str(datetime.now()))

date1 = date(2017, 11, 1)
date2 = date(2018, 5, 31)

datos__2017_2018 = Sentinelsat_products(date1, date2, footprint=footprint)
datos__2017_2018.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos__2017_2018.download_products()
print("Finish datetime: ", str(datetime.now()))

### Download 2018-2019 data

In [ ]:
print("Starting datetime:", str(datetime.now()))

date1 = date(2018, 11, 1)
date2 = date(2019, 5, 31)

datos__2018_2019 = Sentinelsat_products(date1, date2, footprint=footprint)
datos__2018_2019.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos__2018_2019.download_products()

print("Finish datetime: ", str(datetime.now()))

### Download 2019-2020 data

In [ ]:
print("Start datetime:", str(datetime.now()))

date1 = date(2019, 11, 1)
date2 = date(2020, 5, 31)

datos__2019_2020 = Sentinelsat_products(date1, date2, footprint=footprint)
datos__2019_2020.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos__2019_2020.download_products()

print("Finish datetime: ", str(datetime.now()))

### Download 2020-2021 data

In [ ]:
date1 = date(2020, 11, 1)
date2 = date(2021, 3, 27)

datos__2020_2021 = Sentinelsat_products(date1, date2, footprint=footprint)
datos__2020_2021.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos__2020_2021.download_products()